model an assignment problem in Python using MIP where:

Workers can split their time fractionally across tasks.
Each worker has certain credentials (skills).
Each task requires a certain number of hours for each credential.
The model should also detect worker shortages if total available hours for a credential are less than required.
Here’s an example formulation using python-mip that captures this:

Python-MIP fractional assignment with credential constraints & shortage detection

In [1]:
from mip import Model, xsum, CONTINUOUS, minimize

In [105]:
import os
import pandas as pd
import re
os.chdir('/Users/chenya68/Documents/GitHub/uo-mip')

df_res = pd.read_excel('data/ddt_res.xlsx',sheet_name = 'Resources', usecols = [1,2,5,6,7,8],skiprows=1) 
df_res.columns = [re.sub('[^A-Za-z0-9Δ]+', '_', element) for element in df_res.columns]
#df_res.head()
print(df_res.columns)

Index(['Name_Last_', 'Name_First_', 'Prim_Utilization', 'Primary_Role',
       'Sec_Utilization', 'Secondary_Role'],
      dtype='object')


/Users/chenya68/anaconda3/envs/mip/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [106]:
#create the set of workers
# Combine first and last names into a new column "Full_Name"
# str.cat() handles string concatenation efficiently
df_res['Full_Name'] = df_res['Name_First_'].str.strip() + ' ' + df_res['Name_Last_'].str.strip()

# Convert the "Full_Name" column to a Python list
workers = df_res['Full_Name'].tolist()
#workers

In [107]:
# Worker available hours, in the future can pull from outlook
df_res['avail'] = df_res['Prim_Utilization'] + df_res['Sec_Utilization']
avail_hours = dict(zip(df_res['Full_Name'], df_res['avail']))
#avail_hours

In [ ]:
# Worker credentials
#read in roles
df_role = pd.read_excel('data/ddt_res.xlsx',sheet_name = 'data_validation_lists', usecols = 'H',skiprows=1) 
df_role.columns = [re.sub('[^A-Za-z0-9Δ]+', '_', element) for element in df_role.columns]
roles = df_role['Role'].tolist()

n_workers = len(df_res)

worker_creds = dict()
for i,w in enumerate(workers):
    worker_creds[w] = dict()
    for role in roles:
        if df_res.loc[i,'Primary_Role'] == role or df_res.loc[i,'Secondary_Role'] == role:
            worker_creds[w][role] = 1
        else:
            worker_creds[w][role] = 0

"\ndf_creds = pd.DataFrame(index=range(n_workers),columns = roles)\nfor role in roles:\n    for i in range(n_workers):\n        if df_res.loc[i,'Primary_Role'] == role or df_res.loc[i,'Secondary_Role'] == role:\n            df_creds.loc[i,role] = 1\n        else:\n            df_creds.loc[i,role] = 0\ndf_creds.index = workers\n"

In [144]:

df_creds = pd.DataFrame(index=range(n_workers),columns = roles)
for role in roles:
    for i in range(n_workers):
        if df_res.loc[i,'Primary_Role'] == role or df_res.loc[i,'Secondary_Role'] == role:
            df_creds.loc[i,role] = 1
        else:
            df_creds.loc[i,role] = 0
df_creds.index = workers
df_creds.to_csv('data/ddt_worker_creds.csv',index=True,header=True) 


In [110]:
#read in programs and corresponding program type
df_program = pd.read_excel('data/ddt_res.xlsx',sheet_name = 'data_validation_lists', usecols = 'J:K',skiprows=1) 
df_program.columns = [re.sub('[^A-Za-z0-9Δ]+', '_', element) for element in df_program.columns]
df_program = df_program.iloc[:15,:]
#df_program
programs = df_program['Programs'].tolist()
dict_program_type = dict(zip(df_program['Programs'], df_program['type']))
dict_program_type

{'MK-1654': 'ai',
 'MK-7962': 'pfs',
 'MK-7240': 'cd',
 'MK-7240 5mL': 'ai',
 'V540A': 'pfs',
 'V503': 'cd',
 'V350': 'ai',
 'MK-2214': 'pfs',
 'MK-1045': 'cd',
 'MK-5475': 'ai',
 'Verona DPI': 'pfs',
 'MK-3000 PFS': 'pfs',
 'MK-3000 kit': 'ai',
 'MK-8748 PFS': 'pfs',
 'MK-8748 kit': 'cd'}

In [112]:
#read in platform_ai
df_ai = pd.read_excel('data/ddt_res.xlsx',sheet_name = 'platform_ai', 
                        usecols = 'D:S',skiprows=2)
df_ai.index = df_ai.iloc[:,0].values
df_ai = df_ai.iloc[:27,1:]
df_ai['MaxValue'] = df_ai.max(axis=1)
print(df_ai.shape)
df_ai.head()

(27, 16)


,Early,Middle,Late,Early.1,Middle.1,Late.1,Early.2,Middle.2,Late.2,Early.3,Middle.3,Late.3,Early.4,Middle.4,Late.4,MaxValue
Early Prog lead,0.2,0.2,0.2,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2
Tech Assmt HF,0.1,0.2,0.2,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2
Tech Assmt Lead,0.2,0.2,0.2,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2
Tech Assmt Feasibility,0.2,0.4,0.4,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.4
Tech Asses Modeling/Simulation,0.1,0.1,0.1,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1


In [113]:
#read in platform_pfs
df_pfs = pd.read_excel('data/ddt_res.xlsx',sheet_name = 'platform_pfs', 
                        usecols = 'D:S',skiprows=2)
df_pfs.index = df_pfs.iloc[:,0].values
df_pfs = df_pfs.iloc[:27,1:]
df_pfs['MaxValue'] = df_pfs.max(axis=1)
print(df_pfs.shape)
df_pfs.head()

(27, 16)


,Early,Middle,Late,Early.1,Middle.1,Late.1,Early.2,Middle.2,Late.2,Early.3,Middle.3,Late.3,Early.4,Middle.4,Late.4,MaxValue
Early Prog lead,0.2,0.2,0.2,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2
Tech Assmt HF,0.1,0.2,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2
Tech Assmt Lead,0.2,0.2,0.2,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2
Tech Assmt Feasibility,0.2,0.2,0.2,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2
Tech Asses Modeling/Simulation,0.1,0.1,0.1,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1


In [114]:
#read in complex_device
df_cd = pd.read_excel('data/ddt_res.xlsx',sheet_name = 'complex_device', 
                        usecols = 'D:S',skiprows=2)
df_cd.index = df_cd.iloc[:,0].values
df_cd = df_cd.iloc[:27,1:]
df_cd['MaxValue'] = df_cd.max(axis=1)
print(df_cd.shape)
df_cd.head()

(27, 16)


,Early,Middle,Late,Early.1,Middle.1,Late.1,Early.2,Middle.2,Late.2,Early.3,Middle.3,Late.3,Early.4,Middle.4,Late.4,MaxValue
Early Prog lead,0.1,0.5,0.5,0.5,0.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5
Tech Assmt HF,0.2,0.4,0.4,0.2,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.4
Tech Assmt Lead,0.5,0.8,1.0,0.6,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
Tech Assmt Feasibility,0.5,0.8,1.0,0.6,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
Tech Asses Modeling/Simulation,0.2,0.2,0.2,0.2,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2


In [146]:
df_cd.to_csv('data/ddt_cd.csv',index=True,header=True) 
df_ai.to_csv('data/ddt_ai.csv',index=True,header=True) 
df_pfs.to_csv('data/ddt_pfs.csv',index=True,header=True) 

In [115]:
# Task credential requirements (hours)
dict_df_task = dict()
dict_df_task['ai'] = df_ai
dict_df_task['pfs'] = df_pfs
dict_df_task['cd'] = df_cd
task_reqs = dict()
#tasks = list(df_task.columns)
tasks = programs
for j,t in enumerate(tasks):
    task_reqs[t] = dict()
    df_task = dict_df_task[dict_program_type[t]]
    for role in roles:
        if role in df_task.index:
            #task_reqs[t][role] = df_task.loc[role,t].item()
            task_reqs[t][role] = df_task.loc[role,'MaxValue'].item()
        else:
            task_reqs[t][role] = 0.0
#task_reqs

In [141]:
import numpy as np

set_roles = set(roles)

m = Model()

# Decision variables: fraction of worker's time assigned to a task
assign = {(w, t): m.add_var(var_type=CONTINUOUS, lb=0) for w in workers for t in tasks}

# Shortage variables for each credential
shortage = {c: m.add_var(var_type=CONTINUOUS, lb=0) for c in set_roles}

# Worker time limits
for w in workers:
    m += xsum(assign[w, t] for t in tasks) <= avail_hours[w]

# Credential coverage constraints
for t in tasks:
    for c in set_roles:
        m += xsum(assign[w, t] * worker_creds[w][c] for w in workers) + shortage[c] >= task_reqs[t][c]

# Objective: minimize total shortage
m.objective = minimize(xsum(shortage[c] for c in shortage))

m.optimize()

# Output results
print("\nAssignments:")
df_assign = pd.DataFrame(index = workers,columns = tasks)
for w in workers:
    for t in tasks:
        if assign[w, t].x > 1e-6:
            #print(f"{w} -> {t}: {assign[w, t].x:.2f} FTE")
            df_assign.loc[w,t] = np.round(assign[w, t].x,4)

#print(df_assign)

print("\nShortages:")
df_short = pd.DataFrame(index = list(set_roles),columns = ['shortage_fte'])
for c in shortage:
    #print(f"Credential {c}: {shortage[c].x:.2f} FTE")
    df_short.loc[c,'shortage_fte'] = np.round(shortage[c].x,4)

Starting solution of the Linear programming problem using Dual Simplex

Clp0024I Matrix will be packed to eliminate 23760 small elements
Coin0506I Presolve 194 (-285) rows, 754 (-159) columns and 1692 (-633) elements
Clp0014I Perturbing problem by 0.001% of 1 - largest nonzero change 0.00098678741 ( 0.098678741%) - largest zero change 9.8587167e-05
Clp0006I 0  Obj 19.15 Primal inf 172.19981 (194)
Clp0000I Optimal - objective value 24.531111
Coin0511I After Postsolve, objective 24.531111, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 24.53111111 - 218 iterations time 0.002, Presolve 0.00

Assignments:

Shortages:


In [142]:
df_assign.to_csv('data/ddt_res_assignment.csv',index=True,header=True)
df_short.to_csv('data/ddt_res_shortage.csv',index=True,header=True) 